# 경주로 건설

소요시간: NaN

작성일시: 24.03.04 23:34:42

출처: https://school.programmers.co.kr/learn/courses/30/lessons/67259

### 접근 방법
1st
- 백트래킹으로 가능한 모든 경로를 탐색하되, 방문한 칸에서의 최소 비용을 저장하여 현재 경로의 비용이 크면 탐색을 중지하는 방법으로 불필요한 연산을 줄이고자 했다.
- 시간 초과가 발생했다. 
  - 전역 변수를 사용해서?
  - dfs를 사용하면서 유망한 케이스에 대해서만 가지치기하는 과정이 적어져서?

2nd
- bfs로 바꿔보자.
- 25번 케이스만 실패!
  - input: [[0, 0, 0, 0, 0],[0, 1, 1, 1, 0],[0, 0, 1, 0, 0],[1, 0, 0, 0, 1],[1, 1, 1, 0, 0]]
  - output: 3000
  - 한 칸에 방문한 직후 비용은 작을 수 있어도, 이후 회전 이동을 수행하면 비용이 역전될 수 있기에 방문한 직후의 비용을 기준으로 가지치기하면 결과적으로 더 낮은 비용의 경로가 막힐 수 있음
  - 그러면 최소 비용보다 500 큰 금액보다 작을 경우 탐색을 허용하면 어떨까? 비용 역전의 가능성이 있는 케이스를 허용하는 것임.
  - dp table을 최소비용으로 갱신해야 하는데, 위의 경우를 적용했을 때 최소 비용보다 더 크고 최소비용+500보다 작은 금액으로 갱신되면 결과에 영향을 주지 않을까? 잘 모르겠다.

3rd
- 동일한 칸에 접근해 온 방향이 달라서 비용 역전이 발생하는 것이다. 칸 별, 접근 방향 별 최소 비용을 저장하여 백트래킹을 수행하면 된다.
- 성공. 그러나 접근 방향 별로 최소 비용을 따로 저장하여 불필요한 연산이 다소 포함되는 듯하다.

4th
- 3rd를 최적화했다.
- 접근 방향 별로 최소 비용을 따로 저장하되, 비용 역전이 발생할 수 있는 범위인 +500만큼을 고려하여 계산한다. 즉, 현재 접근 방향의 최소 비용이 해당 칸의 최솟값+500보다 크거나 같으면 비용 역전의 여지가 아예 없으므로 탐색을 중지하는 조건을 추가하여 탐색 횟수를 줄일 수 있다.  

### 시간 복잡도 분석
O(N^2)

### 새로 알게 된 것
회전 이동을 수행할 때 방향 배열의 인덱스로 접근하면 편리하게 계산할 수 있다.  
moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]  
동쪽부터 시계방향으로 방향을 집어넣은 위 배열에서 90도 회전을 감지하려면 각 인덱스를 4로 나눈 나머지를 비교하면 된다.  
180도 전환은 +2 하고 4로 나눈 나머지를 구하자.

### 주의할 점
.

### 기타 코멘트
.


In [ ]:
INF = int(1e9)
answer = INF

def solution(board):
    def dfs(r, c, cost, prevDirection):
        global answer
        if r == N-1 and c == N-1:
            answer = min(answer, cost)
            return
        if cost >= answer:
            return
        visited[r][c] = True
        for dr, dc in UDLR:
            nr, nc = r+dr, c+dc
            if isGridIn(nr, nc) and board[nr][nc] != WALL and not visited[nr][nc]:
                pr, pc = prevDirection
                if pr*dr + pc*dc: # straight
                    dfs(nr, nc, cost+100, (dr, dc))
                else: # corner
                    dfs(nr, nc, cost+600, (dr, dc))
        visited[r][c] = False

    def isGridIn(r, c):
        return 0 <= r < N and 0 <= c < N

    WALL = 1
    UDLR = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    N = len(board)
    visited = [[False] * N for _ in range(N)]
    dfs(0, 0, 0, (1, 1))
    return answer

In [ ]:
from collections import deque

def solution(board):
    def isGridIn(r, c):
        return 0 <= r < N and 0 <= c < N

    def bfs():
        q = deque()
        if board[0][1] != WALL:
            q.append((0, 1, H, 100))
            distance[0][1] = 100
        if board[1][0] != WALL:
            q.append((1, 0, V, 100))
            distance[1][0] = 100
        while q:
            r, c, d, cost = q.popleft()
            for direct in moves[d]:
                dr, dc = direct
                nr, nc = r+dr, c+dc
                newCost = cost + 100 if direct[d] else cost + 600
                if isGridIn(nr, nc) and board[nr][nc] != WALL and distance[nr][nc] >= newCost:
                    distance[nr][nc] = newCost
                    q.append((nr, nc, 0 if dr else 1, newCost))

    WALL = 1
    moves = [
        [(1,0), (-1,0), (0,1), (0,-1)],
        [(0,1), (0,-1), (1,0), (-1,0)]
    ]
    V, H = 0, 1
    N = len(board)
    INF = int(1e9)
    distance = [[INF]*N for _ in range(N)]
    bfs()
    return distance[N-1][N-1]

In [ ]:
from collections import deque

def solution(board):
    def isGridIn(r, c):
        return 0 <= r < N and 0 <= c < N

    def bfs():
        distance[0][0] = [0, 0, 0, 0]
        q = deque()
        if board[0][1] != WALL:
            q.append((0, 1, 0, 100))
            distance[0][1][0] = 100
        if board[1][0] != WALL:
            q.append((1, 0, 1, 100))
            distance[1][0][1] = 100
        while q:
            r, c, d, cost = q.popleft()
            for i in range(4):
                dr, dc = moves[i]
                nr, nc = r+dr, c+dc
                newCost = cost + 100 if d%4 == i%4 else cost + 600
                if isGridIn(nr, nc) and board[nr][nc] != WALL and distance[nr][nc][i] > newCost:
                    distance[nr][nc][i] = newCost
                    q.append((nr, nc, i, newCost))

    WALL = 1
    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    N = len(board)
    INF = int(1e9)
    distance = [[[INF]*4 for __ in range(N)] for _ in range(N)]
    bfs()
    return min(distance[N-1][N-1])

In [ ]:
from collections import deque

def solution(board):
    def isGridIn(r, c):
        return 0 <= r < N and 0 <= c < N

    def bfs():
        distance[0][0] = [0, 0, 0, 0]
        q = deque()
        if board[0][1] != WALL:
            q.append((0, 1, 0, 100))
            distance[0][1][0] = 100
        if board[1][0] != WALL:
            q.append((1, 0, 1, 100))
            distance[1][0][1] = 100
        while q:
            r, c, d, cost = q.popleft()
            for i in range(4):
                dr, dc = moves[i]
                nr, nc = r+dr, c+dc
                newCost = cost + 100 if d%4 == i%4 else cost + 600
                if isGridIn(nr, nc) and board[nr][nc] != WALL and distance[nr][nc][i] > newCost and min(distance[nr][nc]) + 500 > newCost:
                    distance[nr][nc][i] = newCost
                    q.append((nr, nc, i, newCost))

    WALL = 1
    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    N = len(board)
    INF = int(1e9)
    distance = [[[INF]*4 for __ in range(N)] for _ in range(N)]
    bfs()
    return min(distance[N-1][N-1])